In [2]:
import pandas as pd
import funciones

In [22]:
#base de datos con las denuncias

df_maltrato_animal = pd.read_csv(r"C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\clean\categorizado\df_maltrato_animal_actualizado_categorizado", encoding = 'latin')
df_maltrato_animal[["fecha_de_recepcion", "fecha_de_ratificacion", "fecha_de_turno", "fecha_de_admision/radicacion", "fecha_de_conclusion"]] = df_maltrato_animal[["fecha_de_recepcion", "fecha_de_ratificacion", "fecha_de_turno", "fecha_de_admision/radicacion", "fecha_de_conclusion"]].apply(pd.to_datetime, errors = 'coerce')


In [23]:
colonias_matched = pd.read_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\clean\colonias_matched.csv',encoding='utf-8')

In [24]:


df_victimas = pd.read_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\clean\df_victimas_actualizado', encoding='utf-8')


In [25]:
from simpledbf import Dbf5

dbf = Dbf5(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\data\raw\ri_11.dbf')

#base de datos externa, densidad de poblacion
df_pob = dbf.to_dataframe().dropna()


funciones.estandarizacion(df_pob, 'colonia', 'alcaldia')

In [26]:
dbf = Dbf5(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\data\raw\ue_comerciales_colonia.dbf')

#base de datos externa, unidades economicas
df_ue = dbf.to_dataframe().dropna()

funciones.estandarizacion(df_ue, 'nombre', 'alcaldia')


In [27]:
datos_corr = funciones.preparacion_enriquecimiento(df_maltrato_animal, 'alcaldia','colonia','id_denuncia')

In [28]:
funciones.enriquecimiento(datos_corr, df_maltrato_animal,'alcaldia','colonia',['maltrato_fisico'],['maltrato_fisico'],'suma')
funciones.enriquecimiento(datos_corr, df_maltrato_animal,'alcaldia','colonia',['intemperie_o_inmovilidad'],['intemperie_o_inmovilidad'],'suma')
funciones.enriquecimiento(datos_corr, df_maltrato_animal,'alcaldia','colonia',['subalimentacion'],['subalimentacion'],'suma')

In [29]:
funciones.enriquecimiento(datos_corr, df_pob, 'alcaldia','colonia', ['SUP_COL_M2', 'pob_2010'],['superficie_metros_cuadrados', 'poblacion_2010'], False)
funciones.enriquecimiento(datos_corr, df_ue, 'alcaldia','nombre', ['ue', 'ind','com','serv'],['unidades_economicas','industriales','comerciales','servicios'], False)

In [30]:
datos_corr['densidad']=(datos_corr['poblacion_2010'] / datos_corr['superficie_metros_cuadrados'])

In [32]:
funciones.enriquecimiento(datos_corr, df_victimas,'alcaldiahechos','coloniahechos',['idcarpeta'], ['num_carpetas'], 'conteo')
funciones.enriquecimiento(datos_corr, df_victimas[df_victimas['delito']=='MALTRATO ANIMAL'], 'alcaldiahechos', 'coloniahechos', ['idcarpeta'], ['num_carpetas_maltrato'], 'conteo')
funciones.enriquecimiento(datos_corr, df_victimas[df_victimas['delito']=='ROBO DE ANIMALES'], 'alcaldiahechos', 'coloniahechos', ['idcarpeta'], ['num_carpetas_robo_animales'], 'conteo')

In [39]:
datos_corr

num_denuncias  maltrato_fisico  \
alcaldia       colonia                                                      
alvaro obregon 1 victoria                              6              0.0   
               1 victoria seccion bosques              3              0.0   
               2 el pirul ampl                         1              0.0   
               2 rios                                  1              0.0   
               8 de agosto                             2              0.0   
...                                                  ...              ...   
xochimilco     tepepan ampl                           18              4.0   
               tierra nueva                            1              0.0   
               tlaxopan                                3              0.0   
               villa xochimilco u hab                  1              0.0   
               xaltocan barr                          15              3.0   

                                           intemperie_o_inmovilidad  \
alcaldia       colonia                                                
alvaro obregon 1 victoria                                       4.0   
               1 victoria seccion bosques                       3.0   
               2 el pirul ampl                                  1.0   
               2 rios                                           1.0   
               8 de agosto                                      2.0   
...                                                             ...   
xochimilco     tepepan ampl                                    13.0   
               tierra nueva                                     1.0   
               tlaxopan                                         3.0   
               villa xochimilco u hab                           1.0   
               xaltocan barr                                   14.0   

                                           subalimentacion  \
alcaldia       colonia                                       
alvaro obregon 1 victoria                              4.0   
               1 victoria seccion bosques              1.0   
               2 el pirul ampl                         1.0   
               2 rios                                  1.0   
               8 de agosto                             2.0   
...                                                    ...   
xochimilco     tepepan ampl                           11.0   
               tierra nueva                            1.0   
               tlaxopan                                3.0   
               villa xochimilco u hab                  0.0   
               xaltocan barr                          13.0   

                                           superficie_metros_cuadrados  \
alcaldia       colonia                                                   
alvaro obregon 1 victoria                                      69993.5   
               1 victoria seccion bosques                      99323.3   
               2 el pirul ampl                                 30139.4   
               2 rios                                          40620.3   
               8 de agosto                                    160837.0   
...                                                                ...   
xochimilco     tepepan ampl                                   764589.0   
               tierra nueva                                   182334.0   
               tlaxopan                                      2272460.0   
               villa xochimilco u hab                          54414.1   
               xaltocan barr                                 1464350.0   

                                           poblacion_2010  \
alcaldia       colonia                                      
alvaro obregon 1 victoria                          2351.0   
               1 victoria seccion bosques          4316.0   
               2 el pirul ampl                     1197.0   
               2 rios                     

In [38]:
datos_corr.to_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\clean\datos_corr_gral.csv')

In [7]:
datos_corr = pd.read_csv(r'C:\Users\samue\OneDrive\Documents\GitHub\captain_planet_sammy\captain-planet\analisis_maltrato_animal\data\clean\datos_corr_gral.csv',encoding='utf-8')


In [8]:
datos_corr

,alcaldia,colonia,num_denuncias,maltrato_fisico,intemperie_o_inmovilidad,subalimentacion,superficie_metros_cuadrados,poblacion_2010,unidades_economicas,industriales,comerciales,servicios,densidad,num_carpetas,num_carpetas_maltrato,num_carpetas_robo_animales
0,alvaro obregon,1 victoria,6,0.0,4.0,4.0,69993.5,2351.0,74.0,5.0,40.0,28.0,0.033589,138.0,NaN,NaN
1,alvaro obregon,1 victoria seccion bosques,3,0.0,3.0,1.0,99323.3,4316.0,110.0,10.0,63.0,35.0,0.043454,NaN,NaN,NaN
2,alvaro obregon,2 el pirul ampl,1,0.0,1.0,1.0,30139.4,1197.0,25.0,1.0,15.0,9.0,0.039715,NaN,NaN,NaN
3,alvaro obregon,2 rios,1,0.0,1.0,1.0,40620.3,1180.0,21.0,3.0,15.0,3.0,0.029050,NaN,NaN,NaN
4,alvaro obregon,8 de agosto,2,0.0,2.0,2.0,160837.0,1704.0,58.0,9.0,24.0,49.0,0.010595,29.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066,xochimilco,tepepan ampl,18,4.0,13.0,11.0,764589.0,8184.0,313.0,22.0,137.0,153.0,0.010704,116.0,NaN,NaN
1067,xochimilco,tierra nueva,1,0.0,1.0,1.0,182334.0,1860.0,142.0,19.0,116.0,162.0,0.010201,67.0,NaN,NaN
1068,xochimilco,tlaxopan,3,0.0,3.0,3.0,2272460.0,4355.0,126.0,14.0,72.0,40.0,0.001916,NaN,NaN,NaN
1069,xochimilco,villa xochimilco u hab,1,0.0,1.0,0.0,54414.1,957.0,43.0,4.0,17.0,22.0,0.017587,8.0,NaN,NaN


In [9]:
datos_corr.groupby('alcaldia').sum()

,num_denuncias,maltrato_fisico,intemperie_o_inmovilidad,subalimentacion,superficie_metros_cuadrados,poblacion_2010,unidades_economicas,industriales,comerciales,servicios,densidad,num_carpetas,num_carpetas_maltrato,num_carpetas_robo_animales
alcaldia,,,,,,,,,,,,,,
alvaro obregon,774,93.0,587.0,522.0,52108248.85,524433.0,20625.5,1341.0,8866.5,9970.5,2.581916,17917.0,20.0,11.0
azcapotzalco,533,65.0,371.0,327.0,27776689.40,317544.0,14394.0,1265.0,6310.0,6904.0,1.315964,13620.0,20.0,7.0
benito juarez,600,72.0,419.0,316.0,18527111.70,264353.0,19253.0,868.0,6512.0,12028.0,0.696230,19126.0,23.0,12.0
coyoacan,671,70.0,473.0,408.0,39072865.00,352361.0,16079.0,939.0,6571.0,9211.0,1.041151,14500.0,26.0,7.0
cuajimalpa de morelos,190,14.0,136.0,119.0,35266951.80,157122.0,7023.0,460.0,3462.0,4002.0,0.217570,4199.0,6.0,3.0
cuauhtemoc,743,88.0,498.0,484.0,18842543.00,285419.0,37700.0,2017.0,20156.0,14259.0,0.634228,39347.0,37.0,20.0
gustavo a madero,1174,132.0,825.0,768.0,54107532.80,737841.0,34726.0,2508.0,16202.0,15346.0,2.264124,28231.0,55.0,8.0
iztacalco,484,53.0,359.0,314.0,10571463.40,172513.0,5958.0,444.0,2466.0,2709.0,0.595743,11766.0,11.0,5.0
iztapalapa,1782,220.0,1315.0,1209.0,75149058.20,1143122.0,56436.5,4606.0,29427.5,21062.0,3.353360,44598.0,44.0,19.0
